# 检查并监控深度学习模型
* 使用keras回调函数 - 访问关于模型状态与性能的所有可用数据
    * 模型检查点:在训练过程中的不同时间点保存模型的当前权重
    * 提前终止：如果验证损失不在改善，则中断训练，同时保存最佳模型
    * 在训练过程中动态调节某些参数
    * 在训练过程中记录训练指标和验证指标，或将模型学到的表示可视化
* Tensorboard - tensorflow的可视化框架


### 1、回调函数 - 在训练过程中将回调函数作用于模型

In [ ]:
# ModelCheckpoint 与 EarlyStopping
import keras
# 通过fit的callback参数将回调函数传入模型中，这个参数接受一个回调函数列表，可以传入任意个数的回调函数
callbacks_list = [
    # 若果不再改善则中断训练
    keras.callbacks.EarlyStopping(
        # 监控模型的验证精度
        monitor = 'acc',
        # 如果验证精度再多一轮的时间内不再改善，中断训练
        patience = 1,
    ),
    # 每轮过后保存当前权重
    keras.callbacks.ModelCheckpoint(
        filepath = 'my_model.h5'
        # 以下两个参数表示如果val_loss没有改善，则不覆盖模型文件，保存最佳模型
        monitor = 'val_loss',
        save_best_only=True,
    )
]
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model.fit(x, y, epochs=10, batch_size=32, callbacks=callbacks_list, validation_data=(x_val,y_val))


# ReduceLROnPlateau  -  若验证损失不再改善则降低学习率
callbacks_list = [
    keras.callbacks.ReduceLROnPlateau(
        # 监控模型的验证损失
        monitor = 'val_loss',
        # 触发式将学习率除以10
        factor=0.1,
        # 若验证损失在10轮内都没有改善，就触发这个回调函数
        patience=10,
    )
]

model.fit(x,y, epochs=10, batch_size=32, callbacks=callbacks_list, validation_data=(x_val,y_val))


# 编写自己的回调函数
'''
    on_epoch_begin   在每轮开始\结束时被调用
    on_eopoch_end   
    on_batch_begin   在处理每个批量之前\后被调用
    on_batch_end
    on_train_begin   在训练之前\后被调用
    on_trian_end
'''
import keras
import numpy as np

class ActivationLogger(keras.callbacks.Callback):
    def set_model(self, model):
        # 在训练之前由父模型调用，告诉回调函数是那个模型在调用它
        self.model = model
        layer_outputs = [layer.output for layer in model.layers]
        # 模型实例，返回每层的激活
        self.activations_model = keras.models.Model(model.input, layer_outputs)
    def on_epoch_end(self, epoch, logs=None):
        if self.validation_data is None:
            raise RuntimeError('Requires validation_data')
        # 获取验证数据的第一个输入样本
        validation_sample = self.validation_data[0][0:1]
        activations = self.activations_model.predict(validation_sample)
        # 将数据保存到硬盘
        f = open('activation_at_epoc_' + str(epoch) + '.npz', 'w')
        np.save(f, activations)
        f.close()

# 2、Tensorboard

In [10]:
import keras
from keras.datasets import imdb
from keras import layers
from keras.preprocessing import sequence

max_features = 2000
max_len = 500

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
x_train = sequence.pad_sequences(x_train, maxlen=max_len)
x_test = sequence.pad_sequences(x_test, maxlen=max_len)

model = keras.models.Sequential()
model.add(layers.Embedding(max_features, 128, input_length=max_len, name='embed'))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.MaxPooling1D(5))
model.add(layers.Conv1D(32, 7, activation='relu'))
# 全局池化层，将三维输出转化为二维输出
model.add(layers.GlobalMaxPooling1D()) 
model.add(layers.Dense(1))

model.summary()
model.compile(optimizer='rmsprop',
              loss = 'binary_crossentropy',
              metrics=['acc'])

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embed (Embedding)            (None, 500, 128)          256000    
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 494, 32)           28704     
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 98, 32)            0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 92, 32)            7200      
_________________________________________________________________
global_max_pooling1d_4 (Glob (None, 32)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 33        
Total params: 291,937
Trainable params: 291,937
Non-trainable params: 0
________________________________________________

In [11]:
callbacks = [
    keras.callbacks.TensorBoard(
        log_dir = 'my_log_dir',
        # 每一轮之后记录激活直方图
        histogram_freq = 1,
        # 每一轮之后记录嵌入数据
        embeddings_freq = 1,
    )
]
history = model.fit(x_train, y_train, 
                    epochs=20, batch_size=128, 
                    validation_split=0.2,
                    callbacks=callbacks)

/home/wukong/.local/lib/python3.6/site-packages/keras/callbacks/tensorboard_v2.py:102: UserWarning: The TensorBoard callback does not support embeddings display when using TensorFlow 2.0. Embeddings-related arguments are ignored.
  warnings.warn('The TensorBoard callback does not support '
/home/wukong/anaconda3/envs/deeplearning/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 20000 samples, validate on 5000 samples


AttributeError: 'Sequential' object has no attribute '_get_distribution_strategy'